In [27]:
#100 nodes variant

from env_v4 import Env_tsp
from config import Config

In [28]:
cfg = Config()
env = Env_tsp(cfg)

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim

class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=64
        )
        self.encoder_output_layer = nn.Linear(
            in_features=64, out_features=64
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=64, out_features=64
        )
        self.decoder_output_layer = nn.Linear(
            in_features=64, out_features=kwargs["input_shape"]
        )
        
    def encode(self, x):
        activation = self.encoder_hidden_layer(x)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        return code
    
    def decode(self, code):
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

    def forward(self, features):
        encoded = self.encode(features)
        reconstructed = self.decode(tmp)
        return reconstructed

In [30]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=2).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [31]:
from torch.utils.data import Dataset, DataLoader

class DistMatrixDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_list, root_dir="", transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = data_list
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if self.transform:
            sample = self.transform(sample)

        return self.data[idx]

In [32]:
import torch.utils.data as data
from torch.utils.data.dataset import random_split

batch = env.get_batch_nodes(50000)

dataset = DistMatrixDataset(batch)
loader = DataLoader(dataset)

# Random split
train_set, valid_set = random_split(dataset, [45000, 5000])

print(len(train_set))
print(len(valid_set))

45000
5000


In [33]:
train_loader = DataLoader(
    train_set, batch_size=128, shuffle=True, num_workers=4, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    valid_set, batch_size=32, shuffle=False, num_workers=4
)

In [34]:
epochs = 50

for epoch in range(epochs):
    loss = 0
    for batch_features in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1,2).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
        if epoch == 48:
            print(batch_features[0])
            print(outputs[0])
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    

    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))
    
    


epoch : 1/50, loss = 0.023611
epoch : 2/50, loss = 0.000016
epoch : 3/50, loss = 0.000006
epoch : 4/50, loss = 0.000003
epoch : 5/50, loss = 0.000002
epoch : 6/50, loss = 0.000002
epoch : 7/50, loss = 0.000001
epoch : 8/50, loss = 0.000001
epoch : 9/50, loss = 0.000002
epoch : 10/50, loss = 0.000003
epoch : 11/50, loss = 0.000002
epoch : 12/50, loss = 0.000003
epoch : 13/50, loss = 0.000003
epoch : 14/50, loss = 0.000004
epoch : 15/50, loss = 0.000003
epoch : 16/50, loss = 0.000003
epoch : 17/50, loss = 0.000004
epoch : 18/50, loss = 0.000003
epoch : 19/50, loss = 0.000004
epoch : 20/50, loss = 0.000003
epoch : 21/50, loss = 0.000004
epoch : 22/50, loss = 0.000003
epoch : 23/50, loss = 0.000004
epoch : 24/50, loss = 0.000003
epoch : 25/50, loss = 0.000004
epoch : 26/50, loss = 0.000004
epoch : 27/50, loss = 0.000003
epoch : 28/50, loss = 0.000003
epoch : 29/50, loss = 0.000003
epoch : 30/50, loss = 0.000003
epoch : 31/50, loss = 0.000004
epoch : 32/50, loss = 0.000003
epoch : 33/50, lo

In [35]:
torch.save(model.state_dict(), "model/vanilla_100n_64emb.pt")